# mcp walkthrough（细讲版）

## 目标与先修
- 目标：理解 MCP 审批流程中 `approval_request -> approval_response -> final_answer` 的分支行为。
- 先修：理解审批流、分支逻辑、JSON 字段读取。


## 流程总览
1. 生成 MCP tool 配置。
2. 请求 `approval_request` 并提取 request id。
3. 分别执行 `approve=True/False` 两条分支。
4. 对比 trace 与最终回答差异。


### 环境检查（在线模块）
- 本步做什么：检查 `DEEPSEEK_API_KEY` 是否存在。
- 为什么这样做：`mcp` 依赖在线 LLM API；缺 key 时必须显式失败，避免误以为流程成功。
- 输入：环境变量 `DEEPSEEK_API_KEY`。
- 输出：通过则继续执行；失败则抛出 `RuntimeError`。
- 观察点：失败信息要明确提示“这是配置问题，不是业务逻辑问题”。


In [ ]:
import os

api_key = os.getenv("DEEPSEEK_API_KEY", "")
if not api_key:
    raise RuntimeError("缺少 DEEPSEEK_API_KEY：在线 walkthrough 不能继续执行。")

print("环境检查通过：已检测到 DEEPSEEK_API_KEY")


### 工具单元（统一 helper）
- 本步做什么：定义 `show_json` 与 `show_kv`，用于稳定展示中间对象。
- 为什么这样做：教学时必须让过程可观测，避免“函数跑完但看不到内部结构”。
- 输入：任意 Python 对象。
- 输出：可读的 JSON 字符串或键值对列表。
- 观察点：后续每步都复用这两个 helper，输出格式保持一致。


In [ ]:
import json


def show_json(obj):
    print(json.dumps(obj, ensure_ascii=False, indent=2, default=str))


def show_kv(title, mapping):
    print(title)
    for key, value in mapping.items():
        print(f"- {key}: {value}")


## 步骤拆解（逐步）

### Step 1: 导入函数并准备输入
- 本步做什么：导入配置、请求与 demo 函数，并定义用户问题。
- 为什么这样做：确保后续两条审批分支使用同一输入。
- 输入：`user_text`。
- 输出：待审批的问题文本。
- 观察点：同一输入下，审批决策将直接改变最后回答。


In [ ]:
from component import build_mcp_tool_config, request_mcp_approval_request, run_demo

user_text = "什么是 Model Context Protocol？"
print(user_text)


### Step 2: 构造 MCP tool 配置
- 本步做什么：调用 `build_mcp_tool_config()`。
- 为什么这样做：审批请求需要明确工具元信息和审批上下文。
- 输入：可选 `server_label`（此处使用默认值）。
- 输出：tool 配置字典。
- 观察点：确认配置中包含 MCP 审批所需字段。


In [ ]:
tool_config = build_mcp_tool_config()
show_json(tool_config)


### Step 3: 请求 approval_request
- 本步做什么：调用 `request_mcp_approval_request(user_text, tool_config)`。
- 为什么这样做：先拿到审批请求对象，再决定 approve/reject 分支。
- 输入：`user_text` + `tool_config`。
- 输出：审批请求字典（含 `approval_request_id`）。
- 观察点：`approval_request_id` 是后续审批响应的关联锚点。


In [ ]:
approval_request = request_mcp_approval_request(user_text, tool_config)
show_json(approval_request)

approval_request_id = str(approval_request["approval_request_id"])
print("approval_request_id:", approval_request_id)


### Step 4: 执行 approve=True 分支
- 本步做什么：运行 `run_demo(user_text, approve=True)`。
- 为什么这样做：观察“审批通过”路径下的 trace 和最终回答。
- 输入：`user_text` 与 `approve=True`。
- 输出：`approved_result`。
- 观察点：trace 中应出现“审批通过”相关事件。


In [ ]:
approved_result = run_demo(user_text, approve=True)
show_json(approved_result)


### Step 5: 执行 approve=False 分支
- 本步做什么：运行 `run_demo(user_text, approve=False)`。
- 为什么这样做：对照“审批拒绝”路径对最终回答的影响。
- 输入：`user_text` 与 `approve=False`。
- 输出：`rejected_result`。
- 观察点：trace 中应出现拒绝分支事件，最终回答语气/内容会变化。


In [ ]:
rejected_result = run_demo(user_text, approve=False)
show_json(rejected_result)


## 端到端结果

### Step 6: 对比两条分支的关键差异
- 本步做什么：提取两条分支的 `final_answer` 与 trace 事件名进行对比。
- 为什么这样做：教学重点是“审批决策如何影响执行路径与输出”。
- 输入：`approved_result`、`rejected_result`。
- 输出：差异对照信息。
- 观察点：对比事件序列和最终回答差异，而不仅是单一字段。


In [ ]:
comparison = {
    "approved_final_answer": approved_result["final_answer"],
    "rejected_final_answer": rejected_result["final_answer"],
    "approved_events": [event["event"] for event in approved_result["trace"]],
    "rejected_events": [event["event"] for event in rejected_result["trace"]],
}
show_json(comparison)


## 常见错误
- 缺少 `DEEPSEEK_API_KEY`：在线调用无法开始。
- `approval_request_id` 缺失：模型返回结构异常，需要排查接口响应。
- approve/reject 结果一致：通常是提示词或分支处理逻辑未生效。

## 总结
- MCP 审批教学要点是“同一请求在不同审批决策下走不同链路”。
- 最有价值的观察对象是 `approval_request_id` 与 trace 事件序列。
